In [ ]:
pip install webdriver_manager

In [1]:
import requests 
import re
from bs4 import BeautifulSoup 
import pandas as pd
import pymongo 
from nltk.corpus import stopwords
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
from config import indeed_query

In [2]:
executable_path = {'executable_path': ChromeDriverManager().install()}



====== WebDriver manager ======
Current google-chrome version is 90.0.4430
Get LATEST driver version for 90.0.4430
Driver [C:\Users\timko\.wdm\drivers\chromedriver\win32\90.0.4430.24\chromedriver.exe] found in cache


In [3]:
 import nltk
 nltk.download('stopwords')
stop_words = set(stopwords.words('english'))  

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\timko\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn) 
db = client.indeed
collection = db.posts

base_url='https://www.indeed.com/'
qloc='Washington+DC'
query=indeed_query
query = query.replace(" ", "%20")
query

'C%23+Net+-clearance+%24101%2C000'

In [8]:

for i in range(0,200,10):
    url = f"{base_url}jobs?q={query}&l={qloc}&jt=fulltime&start={i}"
    print(url)
    #break;
    soup = BeautifulSoup(requests.get(url).text, "html.parser")  
    
    els = soup.select('.jobsearch-SerpJobCard') 
    if len(els) < 1:
        break
    for e in els: 
        if len(e) < 1:
            break
        dic = {}

        dic['query']=query
        dic['qloc']=qloc
        dic['title']=e.select('.title')[0].text.replace('\n','')
        dic['company']=e.select('.company')[0].text.replace('\n','')
        dic['location']=e.select('.location')[0].text.replace('\n','')
        dic['summary']=e.select('.summary')[0].text.replace('\n','')
        dic['entry']='-'.join([c for c in re.findall('\w*', f"{dic['title']}{dic['company']}") if c != '' and c not in stop_words])
        dic['date']=e.select('.date')[0].text.replace('\n','')
        if len(e.select('.salaryText')) > 0: 
            salaryText=e.select('.salaryText')[0].text.replace('\n','')
            dic['salary'] = re.findall('\d{2,3}\,\d{1,3}',salaryText) 
            if len(dic['salary']) > 1:
                dic['salarymin'] =  dic['salary'][0]
                dic['salarymax'] =  dic['salary'][1] 
        collection.delete_one(  {'entry': dic['entry']}  )
        collection.insert_one(dic) 
 


https://www.indeed.com/jobs?q=C%23+Net+-clearance+%24101%2C000&l=Washington+DC&jt=fulltime&start=0
https://www.indeed.com/jobs?q=C%23+Net+-clearance+%24101%2C000&l=Washington+DC&jt=fulltime&start=10
https://www.indeed.com/jobs?q=C%23+Net+-clearance+%24101%2C000&l=Washington+DC&jt=fulltime&start=20
https://www.indeed.com/jobs?q=C%23+Net+-clearance+%24101%2C000&l=Washington+DC&jt=fulltime&start=30
https://www.indeed.com/jobs?q=C%23+Net+-clearance+%24101%2C000&l=Washington+DC&jt=fulltime&start=40
https://www.indeed.com/jobs?q=C%23+Net+-clearance+%24101%2C000&l=Washington+DC&jt=fulltime&start=50
https://www.indeed.com/jobs?q=C%23+Net+-clearance+%24101%2C000&l=Washington+DC&jt=fulltime&start=60
https://www.indeed.com/jobs?q=C%23+Net+-clearance+%24101%2C000&l=Washington+DC&jt=fulltime&start=70
https://www.indeed.com/jobs?q=C%23+Net+-clearance+%24101%2C000&l=Washington+DC&jt=fulltime&start=80
https://www.indeed.com/jobs?q=C%23+Net+-clearance+%24101%2C000&l=Washington+DC&jt=fulltime&start=90
h

In [9]:
df = pd.DataFrame(collection.find({}))
try:
    df['salarymin'] = df['salarymin'].apply(lambda x: str(x).replace(',', '')).astype('float') 
except:
    pass
df['url'] = 'https://www.google.com/search?q=' + df['company'] + '+Careers'
df['url'] = df['url'].apply(lambda x: str(x).replace(' ', '%20'))
try:
    df['date'] = df['date'].apply(lambda x: re.sub('days ago|day ago|\+|Today|\s', '', x)).replace('','0').astype('float') 
except:
    pass

df.sort_values(by='date', ascending=True, inplace=True) 
#df = df.loc[df.title.str.contains('Developer|Engineer|Software', regex=True)]
for i, r in df.iterrows():
    print(f"{r['title']} {r['company']} {r['url']}" )
df

Remote Senior Software Engineer, C# / .NETnew Verys https://www.google.com/search?q=Verys+Careers
Full Stack .Net Developer ( US citizen or GC Holder )new CLS Bank International https://www.google.com/search?q=CLS%20Bank%20International+Careers
.NET Developer (US Citizen)new Armedia LLC https://www.google.com/search?q=Armedia%20LLC+Careers
Business Central Senior Engineernew Wipfli LLP https://www.google.com/search?q=Wipfli%20LLP+Careers
Software Engineer PolyCon Solutions https://www.google.com/search?q=PolyCon%20Solutions+Careers
Senior .Net Applications Developer SOFTEK https://www.google.com/search?q=SOFTEK+Careers
Full Stack Software Engineer (.NET/ReactJS) B-Line Medical https://www.google.com/search?q=B-Line%20Medical+Careers
Sr. Microsoft SharePoint Developer NFF Inc https://www.google.com/search?q=NFF%20Inc+Careers
Microsoft Dynamics 365 Developer (OIM-00045-2019) National Heart, Lung, and Blood Institute https://www.google.com/search?q=National%20Heart,%20Lung,%20and%20Blood%

,_id,query,qloc,title,company,location,summary,entry,date,salary,salarymin,salarymax,url
3,6088b6269100378cd256e88c,C%23+Net+-clearance+%24101%2C000,Washington+DC,"Remote Senior Software Engineer, C# / .NETnew",Verys,United States,-We are unable to provide sponsorship at this ...,Remote-Senior-Software-Engineer-C-NETnewVerys,1 day ago,NaN,NaN,NaN,https://www.google.com/search?q=Verys+Careers
4,6088b6269100378cd256e88d,C%23+Net+-clearance+%24101%2C000,Washington+DC,Full Stack .Net Developer ( US citizen or GC H...,CLS Bank International,"Herndon, VA","Dynamics 365, C#, XAML, JavaScript, Spring Boo...",Full-Stack-Net-Developer-US-citizen-GC-Holder-...,1 day ago,[],NaN,NaN,https://www.google.com/search?q=CLS%20Bank%20I...
13,6088b6269100378cd256e89c,C%23+Net+-clearance+%24101%2C000,Washington+DC,.NET Developer (US Citizen)new,Armedia LLC,"Vienna, VA 22182",We are a CMMI Level 3 systems integrator provi...,NET-Developer-US-Citizen-newArmedia-LLC,1 day ago,NaN,NaN,NaN,https://www.google.com/search?q=Armedia%20LLC+...
91,6088b62b9100378cd256e909,C%23+Net+-clearance+%24101%2C000,Washington+DC,Business Central Senior Engineernew,Wipfli LLP,"Reston, VA 20191 (Hattontown area)","Configure, develop and deliver MS Dynamics Bus...",Business-Central-Senior-EngineernewWipfli-LLP,1 day ago,NaN,NaN,NaN,https://www.google.com/search?q=Wipfli%20LLP+C...
110,6088b62d9100378cd256e93f,C%23+Net+-clearance+%24101%2C000,Washington+DC,Software Engineer,PolyCon Solutions,"Washington, DC",PolyCon Solutions is seeking an experienced So...,Software-EngineerPolyCon-Solutions,11 days ago,NaN,NaN,NaN,https://www.google.com/search?q=PolyCon%20Solu...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,6088b62b9100378cd256e90b,C%23+Net+-clearance+%24101%2C000,Washington+DC,Senior Software Developer,Ledgent Technology,"Washington, DC",You will be responsible for turning research c...,Senior-Software-DeveloperLedgent-Technology,Active 8 days ago,"[110,000, 150,000]",110000.0,"150,000",https://www.google.com/search?q=Ledgent%20Tech...
118,6088b6309100378cd256e97c,C%23+Net+-clearance+%24101%2C000,Washington+DC,Application Developer (Remote Flexibility)new,TEKsystems,"Arlington, VA 22201","5+ years of C#, MVC, and .NET Core experience ...",Application-Developer-Remote-Flexibility-newTE...,Just posted,NaN,NaN,NaN,https://www.google.com/search?q=TEKsystems+Car...
8,6088b6269100378cd256e896,C%23+Net+-clearance+%24101%2C000,Washington+DC,SharePoint Developer/Administratornew,"OneSource IT, Inc.","Washington, DC",The SharePoint Developer/Administrator will be...,SharePoint-Developer-AdministratornewOneSource...,Just posted,"[90,000, 120,000]",90000.0,"120,000",https://www.google.com/search?q=OneSource%20IT...
12,6088b6269100378cd256e89b,C%23+Net+-clearance+%24101%2C000,Washington+DC,SharePoint Administrator/Developernew,ITCON Services,"Washington, DC 20024 (Southwest Waterfront area)",This opening is for a full-time salaried posit...,SharePoint-Administrator-DevelopernewITCON-Ser...,Today,"[79,553, 135,092]",79553.0,"135,092",https://www.google.com/search?q=ITCON%20Servic...


In [ ]:
conn = 'mongodb://localhost:27017' 
cursor = collection.find(
    { }, {'_id': False})
df = pd.DataFrame(cursor)
df

In [ ]:
 
collection.delete_many(
     { 'entry': {'$regex': '^.*$', '$options': 'i'}}
)
 

In [ ]:
browser = Browser('chrome', **executable_path, headless=False)

base_url='https://www.indeed.com/'
qloc='Washington+DC'
query='junior+data+python'
for i in range(0,20,10): 
    browser.visit(f"{base_url}jobs?q={query}&l={qloc}&jt=fulltime&start={i}")
    soup = BeautifulSoup( browser.html , 'html.parser') 
    for ele in soup.select('.jobsearch-SerpJobCard a.jobtitle'): 
        browser.click_link_by_id(f"{ele['id']}")

        with browser.get_iframe('vjs-container-iframe') as iframe: 
            soup = BeautifulSoup( iframe.html , 'html.parser')
            print(soup.select('#jobDescriptionText') == None)  